In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hvv-preprocess/input_val.jsonl
/kaggle/input/hvv-preprocess/input_train.jsonl
/kaggle/input/hvv-meme/constraint22_dataset_uspolitics/images/memes_3335.png
/kaggle/input/hvv-meme/constraint22_dataset_uspolitics/images/memes_1208.png
/kaggle/input/hvv-meme/constraint22_dataset_uspolitics/images/memes_4967.png
/kaggle/input/hvv-meme/constraint22_dataset_uspolitics/images/memes_1796.png
/kaggle/input/hvv-meme/constraint22_dataset_uspolitics/images/memes_4673.png
/kaggle/input/hvv-meme/constraint22_dataset_uspolitics/images/memes_4754.png
/kaggle/input/hvv-meme/constraint22_dataset_uspolitics/images/memes_1260.png
/kaggle/input/hvv-meme/constraint22_dataset_uspolitics/images/memes_4875.png
/kaggle/input/hvv-meme/constraint22_dataset_uspolitics/images/memes_835.png
/kaggle/input/hvv-meme/constraint22_dataset_uspolitics/images/memes_3834.png
/kaggle/input/hvv-meme/constraint22_dataset_uspolitics/images/memes_322.png
/kaggle/input/hvv-meme/constraint22_dataset_uspolitics/images/m

In [2]:
import numpy as np
import pandas as pd
import json
import gzip
import pickle
# import dgl 
import os
import random
import torch
from sklearn.utils import resample
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Optional, Tuple, Union
from collections.abc import Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

2024-05-07 16:05:55.954996: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 16:05:55.955056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 16:05:55.956562: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Hyperparameters

In [3]:
# %% HyperParameters

DEVICE = ''

if torch.cuda.is_available():
    DEVICE = torch.device("cuda:0")
#     DEVICE = torch.device("cuda:1")
    print("Using GPU: ", DEVICE)
else:
    DEVICE = torch.device("cpu")
    print("Using CPU: ", DEVICE)
    
model_base = 'distilbert/distilbert-base-uncased'
ID2LABEL = ['hero', 'villain', 'victim', 'other']
LABEL2ID = {'hero':0, 'villain':1, 'victim':2, 'other':3}
TOKENIZER = AutoTokenizer.from_pretrained(model_base)
DATA_COLLATOR = DataCollatorWithPadding(tokenizer=TOKENIZER)
LM = AutoModelForSequenceClassification.from_pretrained(
    model_base, num_labels=len(ID2LABEL), id2label=ID2LABEL, label2id=LABEL2ID
).to(DEVICE)

TRAIN_DIR = '/kaggle/input/hvv-preprocess/input_train.jsonl'
TEST_DIR = '/kaggle/input/hvv-preprocess/input_val.jsonl'
VAL_DIR = '/kaggle/input/hvv-preprocess/input_val.jsonl'
OUTPUT_DIR = '/kaggle/working/bert_'

LR = 1e-6
TR_BATCH = 32
EV_BATCH = 32
EPOCH = 15
WEIGHT_DECAY = 0.01
UPSAMPLES = 2000
DATA_AUGMENTATION = True
LM_MAX_LEN = 512

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=LR,
    per_device_train_batch_size=TR_BATCH,
    per_device_eval_batch_size=EV_BATCH,
    num_train_epochs=EPOCH,
    weight_decay=WEIGHT_DECAY,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)



Using GPU:  cuda:0


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Dataloader

In [4]:

def expand_entity_data(json_data):
    original = []
    text = []
    image_name = []
    word = []
    role = []
#     meme_context = []
    
    for vals in tqdm(json_data):
        
        ocr = vals['OCR']
        if ocr is not None:
            ocr = ocr.lower().replace('\n', ' ')
        else:
            ocr = None
        
        image = vals['image'].lower()
        
        for keys in ['hero', 'villain', 'victim', 'other']:
            for word_val in vals[keys]:
                original.append(vals['OCR'])
                text.append(ocr)
                word.append(word_val)
                role.append(keys)
                image_name.append(image)
#                 meme_context.append(context)

    df = pd.DataFrame()

    df['sentence'] = text
    df['original'] = original 
    df['image'] = image_name
    df['entity'] = word 
    df['entity_type'] = role
    df['label'] = [LABEL2ID[r] for r in role]
    df['sentence'] = df['sentence'].apply(lambda x : str(x) + ' [SEP] ')
    df['sentence'] = df['sentence'] + df['entity']
#     df['context'] = meme_context

    return df

def get_total_dataset(augmentation='no'):

    print("\nLoading CONSTRAINT Dataset...\n")
    
    # ----------------------------------------------- READ TRAIN DATA -----------------------------------------------
    
    json_data_train = []
    
    with open(TRAIN_DIR, 'r') as json_file:
        for json_str in list(json_file):
            json_data_train.append(json.loads(json_str))
    json_file.close()
    
    df_train = expand_entity_data(json_data_train)
    

    # Up sampling of the minority classes  
    if augmentation == 'yes':
        
        df_other = df_train[df_train.entity_type=='other']
        df_hero = df_train[df_train.entity_type=='hero']
        df_villian = df_train[df_train.entity_type=='villain']
        df_victim = df_train[df_train.entity_type=='victim']
    
        df_hero_upsampled = resample(df_hero, 
                                     replace=True,     # sample with replacement
                                     n_samples=UPSAMPLES,    # to match majority class
#                                      random_state=4) # reproducible results
                                     random_state=42) # reproducible results

        df_other = pd.concat([df_other, df_hero])
        df_other = pd.concat([df_other, df_hero_upsampled])

        df_villian_upsampled = resample(df_villian, 
                                        replace=True,     # sample with replacement
                                        n_samples=UPSAMPLES,    # to match majority class
#                                         random_state=4) # reproducible results
                                        random_state=42) # reproducible results

        df_other = pd.concat([df_other, df_villian])
        df_other = pd.concat([df_other, df_villian_upsampled])

        df_victim_upsampled = resample(df_victim, 
                                        replace=True,     # sample with replacement
                                        n_samples=UPSAMPLES,    # to match majority class
#                                        random_state=4) # reproducible results
                                        random_state=42) # reproducible results

        df_train_final = pd.concat([df_other, df_victim])
        df_train_final = pd.concat([df_train_final, df_victim_upsampled])
#         df_train_final = df_train_final.reset_index(drop_index=False)
        
    else:
        df_train_final = df_train
    
    df_train_final['ID'] = np.arange(len(df_train_final))
    df_train_final['ID'] = df_train_final['ID'].apply(lambda x: 'train_' + str(x))
    
    # ----------------------------------------------- READ VAL DATA -----------------------------------------------
    
    json_data_val = []

    # if USE_COVID:
    with open(VAL_DIR, 'r') as json_file:
        for json_str in list(json_file):
            json_data_val.append(json.loads(json_str))
    json_file.close()
    
    df_val = expand_entity_data(json_data_val)
    
    df_val['ID'] = np.arange(len(df_val))
    df_val['ID'] = df_val['ID'].apply(lambda x: 'val_' + str(x))
    
    # ----------------------------------------------- READ TEST DATA -----------------------------------------------
    
    json_data_test = []
    
    df_test = expand_entity_data(json_data_test)
    
    return df_train_final, df_val, df_test


## Dataset

In [5]:
class HVVCustomDataset(Dataset):
    
    def __init__(
        self,
        df: pd.DataFrame,
        data_type: str,
        lm_tokenizer,
    ): 
        self.df = df
        print("\nDataset length: ", self.df.shape)
        print("\nLabel Distribution: \n", self.df['label'].value_counts())
        self.data_type = data_type
        self.lm_tokenizer = lm_tokenizer
        
    def __len__(
        self
    ):
        return len(self.df)

    def __getitem__(
        self,
        index: int
    ):
        data_point = self.df.iloc[index]
        ID = str(data_point['ID'])
            
        # context = "[MEME ENTITY]" + str(data_point['entity'])
        
        text_inputs = self.lm_tokenizer.encode_plus(
            str(data_point['sentence']),
#             context,
            max_length=LM_MAX_LEN,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_token_type_ids=True,            
            return_tensors='pt'
        )
            
            
#         text_inputs = data_point['sentence'] + ' [SEP] ' + data_point['entity']
        
        data_inputs = {
            'input_ids': text_inputs['input_ids'].flatten(),
            'attention_mask': text_inputs['attention_mask'].flatten(),
            'token_type_ids': text_inputs["token_type_ids"].flatten(),
            'label': torch.tensor(self.df.iloc[index]['label'], dtype=torch.int)
        }
                           
        return data_inputs

## Loading Dataset

In [6]:
train_data, val_data, test_data = get_total_dataset(augmentation=DATA_AUGMENTATION)

train_dataset = HVVCustomDataset(
        df=train_data, 
        data_type='train', 
        lm_tokenizer=TOKENIZER
    )
train_loader = DataLoader(train_dataset, batch_size=TR_BATCH, 
                          num_workers=4, shuffle=True, pin_memory=True)

print("\n\nTrain data loaded, length:", len(train_dataset))

val_dataset = HVVCustomDataset(
        df=val_data,
        data_type='val',
        lm_tokenizer=TOKENIZER
    )


valid_loader = DataLoader(val_dataset, batch_size=EV_BATCH, 
                          num_workers=4, shuffle=False, pin_memory=True)

print("\n\nVal data loaded, length:", len(val_dataset))



Loading CONSTRAINT Dataset...



100%|██████████| 271/271 [00:00<00:00, 203555.94it/s]
0it [00:00, ?it/s]


Dataset length:  (6419, 7)

Label Distribution: 
 label
3    5323
1     605
2     331
0     160
Name: count, dtype: int64


Train data loaded, length: 6419

Dataset length:  (765, 7)

Label Distribution: 
 label
3    629
1     73
2     43
0     20
Name: count, dtype: int64


Val data loaded, length: 765


## Trainer and Tester

In [7]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [8]:
from tqdm.auto import tqdm
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def train(model, criterion, train_loader, valid_loader):
    progress_bar_train = tqdm(range(num_training_steps), desc="Training")
    progress_bar_eval = tqdm(range(num_eval_steps), desc="Validation")

    for epoch in range(EPOCH):
        model.train()
        total_train_loss = 0.0
        for _, data in enumerate(train_loader, 0):
            ids = data['input_ids'].to(DEVICE, dtype=torch.long)
            mask = data['attention_mask'].to(DEVICE, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(DEVICE, dtype=torch.long)
            labels = data['label'].to(DEVICE, dtype=torch.float)  # Adjust label type

            optimizer.zero_grad()

#             outputs = model(ids, attention_mask=mask, token_type_ids=token_type_ids).logits  # Access logits for classification
            outputs = model(ids, attention_mask=mask).logits  # Access logits for classification
            targets = torch.eye(len(ID2LABEL), device=DEVICE, dtype=torch.float)[labels.long()]
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

            progress_bar_train.update(1)

        avg_train_loss = total_train_loss / len(train_loader)

        model.eval()
        total_predictions = []
        total_labels = []
        total_eval_loss = 0.0

        with torch.no_grad():
            for _, data in enumerate(valid_loader, 0):
                ids = data['input_ids'].to(DEVICE, dtype=torch.long)
                mask = data['attention_mask'].to(DEVICE, dtype=torch.long)
                token_type_ids = data['token_type_ids'].to(DEVICE, dtype=torch.long)
                labels = data['label'].to(DEVICE, dtype=torch.long)  # Adjust label type

#                 outputs = model(ids, attention_mask=mask, token_type_ids=token_type_ids).logits  # Access logits for classification
                outputs = model(ids, attention_mask=mask).logits  # Access logits for classification
                
                targets = torch.eye(len(ID2LABEL), device=DEVICE, dtype=torch.float)[labels.long()]
                predictions = torch.argmax(outputs, dim=1)  # Get the class with the highest probability
                total_predictions.extend(predictions.cpu().tolist())
                total_labels.extend(labels.cpu().tolist())

                progress_bar_eval.update(1)

                eval_loss = criterion(outputs, targets).item()
                total_eval_loss += eval_loss

        accuracy = accuracy_score(total_labels, total_predictions)
        precision = precision_score(total_labels, total_predictions, average='weighted')
        recall = recall_score(total_labels, total_predictions, average='weighted')
        f1 = f1_score(total_labels, total_predictions, average='weighted')
        
        if epoch % 2 == 1:
            model_path = OUTPUT_DIR+'model_epoch_'+str(epoch+1)+'.pth'
            torch.save(model.state_dict(), model_path)
        print(f"Epoch {epoch+1}/{EPOCH}:")
        print(f"Train Loss: {avg_train_loss:.4f} | Validation Metrics: Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}, Loss: {total_eval_loss/len(valid_loader)}")


In [9]:
def test(model, test_loader):
    progress_bar_test = tqdm(range(len(test_loader)), desc="Testing")
    
    with torch.no_grad():
        model.eval()
        total_predictions = []
        total_labels = []
        total_eval_loss = 0.0
        for _, data in enumerate(test_loader, 0):
            ids = data['input_ids'].to(DEVICE, dtype=torch.long)
            mask = data['attention_mask'].to(DEVICE, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(DEVICE, dtype=torch.long)
            labels = data['label'].to(DEVICE, dtype=torch.long)  # Adjust label type

#             outputs = model(ids, attention_mask=mask, token_type_ids=token_type_ids).logits  # Access logits for classification
            outputs = model(ids, attention_mask=mask).logits  # Access logits for classification
            targets = torch.eye(len(ID2LABEL), device=DEVICE, dtype=torch.float)[labels.long()]
            predictions = torch.argmax(outputs, dim=1)  # Get the class with the highest probability
            total_predictions.extend(predictions.cpu().tolist())
            total_labels.extend(labels.cpu().tolist())

            progress_bar_test.update(1)

            eval_loss = criterion(outputs, targets).item()
            total_eval_loss += eval_loss
    
    accuracy = accuracy_score(total_labels, total_predictions)
    precision = precision_score(total_labels, total_predictions, average='weighted')
    recall = recall_score(total_labels, total_predictions, average='weighted')
    f1 = f1_score(total_labels, total_predictions, average='weighted')
    
    print(f"Validation Metrics: Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}, Loss: {total_eval_loss/len(test_loader)}")
    return total_predictions


## Training

In [10]:
from transformers import get_scheduler
from torch.optim import AdamW

# Define optimizer
optimizer = AdamW(LM.parameters(), lr=1e-4)

# Calculate total number of training steps
num_training_steps = EPOCH * len(train_loader)
num_eval_steps = EPOCH * len(valid_loader)

# Define learning rate scheduler
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

criterion = nn.BCEWithLogitsLoss()

In [ ]:
train(model = LM, criterion = criterion, train_loader = train_loader, valid_loader = valid_loader)

Training:   0%|          | 0/3015 [00:00<?, ?it/s]

Validation:   0%|          | 0/360 [00:00<?, ?it/s]

Epoch 1/15:
Train Loss: 0.2771 | Validation Metrics: Accuracy: 0.8222, Precision: 0.6760, Recall: 0.8222, F1-score: 0.7420, Loss: 0.2702119288345178


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2/15:
Train Loss: 0.2403 | Validation Metrics: Accuracy: 0.8078, Precision: 0.7159, Recall: 0.8078, F1-score: 0.7563, Loss: 0.26125466885666054
Epoch 3/15:
Train Loss: 0.2002 | Validation Metrics: Accuracy: 0.8222, Precision: 0.8000, Recall: 0.8222, F1-score: 0.7527, Loss: 0.2948727474237482


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 4/15:
Train Loss: 0.1702 | Validation Metrics: Accuracy: 0.7856, Precision: 0.7395, Recall: 0.7856, F1-score: 0.7588, Loss: 0.28951008183260757
Epoch 5/15:
Train Loss: 0.1376 | Validation Metrics: Accuracy: 0.7739, Precision: 0.7412, Recall: 0.7739, F1-score: 0.7552, Loss: 0.34897091581175727


In [ ]:
# !rm ./model_epoch_*

In [ ]:
# train(model = LM, criterion = criterion, train_loader = train_loader, valid_loader = valid_loader)

# Inference

In [ ]:
# TEST_DIR = '/kaggle/input/hvv-test-unprocessed/test_questions.jsonl'
TEST_DIR = '/kaggle/input/hvv-test-processed/test.jsonl'

def expand_entity_test(json_data):
    original = []
    text = []
    image_name = []
    word = []
    entity_id = []
#     meme_context = []
    
    for vals in tqdm(json_data):
        
        ocr = vals['OCR']
        if ocr is not None:
            ocr = ocr.lower().replace('\n', ' ')
        else:
            ocr = None
        
        image = vals['image'].lower()
        
#         for keys in ['hero', 'villain', 'victim', 'other']:
        for entity_ids, word_val in vals['target_list']:
            original.append(vals['OCR'])
            text.append(ocr)
            word.append(word_val)
            entity_id.append(entity_ids)
            image_name.append(image)
#                 meme_context.append(context)

    df = pd.DataFrame()

    df['sentence'] = text
    df['original'] = original 
    df['image'] = image_name
    df['entity'] = word 
    df['entity_id'] = entity_id
#     df['label'] = [LABEL2ID[r] for r in role]
    df['sentence'] = df['sentence'].apply(lambda x : str(x) + ' [SEP] ')
    df['sentence'] = df['sentence'] + df['entity']
#     df['context'] = meme_context

    return df

In [ ]:
class HVVTestDataset(Dataset):
    
    def __init__(
        self,
        df: pd.DataFrame,
        data_type: str,
        lm_tokenizer,
    ): 
        self.df = df
        print("\nDataset length: ", self.df.shape)
#         print("\nLabel Distribution: \n", self.df['label'].value_counts())
        self.data_type = data_type
        self.lm_tokenizer = lm_tokenizer
        
    def __len__(
        self
    ):
        return len(self.df)

    def __getitem__(
        self,
        index: int
    ):
        data_point = self.df.iloc[index]
        ID = str(data_point['ID'])
            
        # context = "[MEME ENTITY]" + str(data_point['entity'])
        
        text_inputs = self.lm_tokenizer.encode_plus(
            str(data_point['sentence']),
#             context,
            max_length=LM_MAX_LEN,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_token_type_ids=True,            
            return_tensors='pt'
        )
            
            
#         text_inputs = data_point['sentence'] + ' [SEP] ' + data_point['entity']
        
        data_inputs = {
            'input_ids': text_inputs['input_ids'].flatten(),
            'attention_mask': text_inputs['attention_mask'].flatten(),
            'token_type_ids': text_inputs["token_type_ids"].flatten(),
#             'label': torch.tensor(self.df.iloc[index]['label'], dtype=torch.int)
        }
                           
        return data_inputs

In [ ]:
json_data_test = []

with open(TEST_DIR, 'r') as json_file:
    for json_str in list(json_file):
        json_data_test.append(json.loads(json_str))
json_file.close()

df_test = expand_entity_test(json_data_test)

df_test['ID'] = np.arange(len(df_test))
# df_test['ID'] = df_val['ID'].apply(lambda x: 'val_' + str(x))

test_dataset = HVVTestDataset(
        df=df_test,
        data_type='test',
        lm_tokenizer=TOKENIZER
    )


test_loader = DataLoader(test_dataset, batch_size=EV_BATCH, 
                          num_workers=4, shuffle=False, pin_memory=True)

print("\n\nTest data loaded, length:", len(test_dataset))

In [ ]:
def predict(model, test_loader):
    progress_bar_test = tqdm(range(len(test_loader)), desc="Testing")
    
    with torch.no_grad():
        model.eval()
        total_predictions = []
        total_labels = []
        total_eval_loss = 0.0
        for _, data in enumerate(test_loader, 0):
            ids = data['input_ids'].to(DEVICE, dtype=torch.long)
            mask = data['attention_mask'].to(DEVICE, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(DEVICE, dtype=torch.long)
#             labels = data['label'].to(DEVICE, dtype=torch.long)  # Adjust label type

#             outputs = model(ids, attention_mask=mask, token_type_ids=token_type_ids).logits  # Access logits for classification
            outputs = model(ids, attention_mask=mask).logits  # Access logits for classification
#             targets = torch.eye(len(ID2LABEL), device=DEVICE, dtype=torch.float)[labels.long()]
            predictions = torch.argmax(outputs, dim=1)  # Get the class with the highest probability
            total_predictions.extend(predictions.cpu().tolist())
#             total_labels.extend(labels.cpu().tolist())

            progress_bar_test.update(1)

#             eval_loss = criterion(outputs, targets).item()
#             total_eval_loss += eval_loss
    
#     accuracy = accuracy_score(total_labels, total_predictions)
#     precision = precision_score(total_labels, total_predictions, average='weighted')
#     recall = recall_score(total_labels, total_predictions, average='weighted')
#     f1 = f1_score(total_labels, total_predictions, average='weighted')
#     
#     print(f"Validation Metrics: Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}, Loss: {total_eval_loss/len(test_loader)}")
    return total_predictions


In [ ]:
model_path = OUTPUT_DIR+'model_epoch_'+str(4)+'.pth'
mOdel = AutoModelForSequenceClassification.from_pretrained(model_base, num_labels=len(ID2LABEL), id2label=ID2LABEL, label2id=LABEL2ID).to(DEVICE)
mOdel.load_state_dict(torch.load(model_path))

In [ ]:
pred = predict(mOdel, test_loader)

In [ ]:
df = pd.DataFrame({"ID":df_test['entity_id'], "entity":df_test['entity'], 'label':pred})
df['label'] = df['label'].apply(lambda x: ID2LABEL[x].upper())
# df['true'] = df['true'].apply(lambda x: ID2LABEL[x].upper())
df[250:300]

In [ ]:
df = df.drop(columns = ['entity'])
# df.drop(columns = ['ID'], inplace = True)
df.rename(columns={'label': 'ENTITY_LABEL'}, inplace = True)
# df.to_csv('/kaggle/working/unprocessed.csv')
df.to_csv('/kaggle/working/processed.csv', index = False)

In [ ]:
df.head()